# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.234507284560                   -0.50    8.0         
  2   -7.250227207983       -1.80       -1.40    1.0   7.80ms
  3   -7.251230322894       -3.00       -2.12    2.0   9.25ms
  4   -7.251296290621       -4.18       -2.22    2.0   9.72ms
  5   -7.251336994617       -4.39       -2.98    1.0   7.85ms
  6   -7.251337507266       -6.29       -3.09    2.0   9.26ms
  7   -7.251338664877       -5.94       -3.59    1.0   8.08ms
  8   -7.251338778200       -6.95       -3.96    2.0   9.60ms
  9   -7.251338797397       -7.72       -4.60    1.0   8.21ms
 10   -7.251338798634       -8.91       -5.05    2.0   10.0ms
 11   -7.251338798690      -10.25       -5.52    1.0   8.31ms
 12   -7.251338798703      -10.87       -6.05    2.0   10.0ms
 13   -7.251338798704      -12.05       -6.36    2.0   9.91ms
 14   -7.251338798705      -12.84       -6.88    1.0   8.35ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.038577109071114675
[ Info: Arnoldi iteration step 2: normres = 0.5071738245665345
[ Info: Arnoldi iteration step 3: normres = 0.901488342819773
[ Info: Arnoldi iteration step 4: normres = 0.3499786669656136
[ Info: Arnoldi iteration step 5: normres = 0.35492471295912315
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (4.68e-02, 4.90e-02, 2.70e-01, 2.19e-01, 1.62e-02)
[ Info: Arnoldi iteration step 6: normres = 0.48044800423603573
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.91e-02, 1.86e-01, 4.08e-01, 1.19e-01, 9.79e-02)
[ Info: Arnoldi iteration step 7: normres = 0.08966012072873142
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (9.50e-04, 3.29e-02, 1.28e-02, 4.58e-02, 5.82e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10148503267546882
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (4.09e